In [1]:
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('df-project').getOrCreate()
print(spark)
sc = spark.sparkContext
print(sc)

<SparkContext master=local[*] appName=df-project>


In [4]:
#Working with ROWs
from pyspark.sql import Row
row = Row(name="Robert",age=34)
type(row)
print("Accessing Name property from Row object :" + row.name)

#Using in operator
"name" in row 
"name1" in row 
"Robert" in row.name
"Robert1" in row.name


Accessing Name property from Row object :Robert


True

In [5]:
# Creating the RDD using the ROWs function 
lst=[Row(name="Robert",age=34),Row(name="alicia",age=45)]
schema = ["Name","Age"]

rdd = sc.parallelize(lst)
for i in rdd.take(2) : print(i.name)

Robert
alicia


In [5]:
#Using Custom Row 
Person = Row("Name","Age")
p1 = Person("Robert",34)
p2 = Person("alicia",56)
#Accessing the object values 
print(p1.Name)
print(p1.Age)

#Creating the RDD using Custom Rows
lst=[Person("Robert",34),Person("alicia",38)]
rdd = sc.parallelize(lst)
for i in rdd.take(2):print(i.Name)


Robert
34
Robert
alicia


In [6]:
#Using few Row functions
Person=Row(Name="Alice",age=34,username="Alice")
Person.count('Alice')
Person.count(34)

#index
Person.index(34)
Person

Row(Name='Alice', age=34, username='Alice')

In [7]:
df = spark.read.load('D:\SpectraMD\Feed\Benchmarks',format='csv',sep='|',header=True)
df.show()
df.printSchema()

# accessing the columns 
df.select(df.Year).show()
#other Way 
df.select(df["Year"]).show()

#Using col function
from pyspark.sql.functions import col
df.select(col("*")).show() # To access all the columns in dataframe
df.select(col("Year")).show() # To access specific column in datafram



+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+--------------+-----------------+-------------+-------------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|95thPercentile|   base_measureid|sub_measureid|   base_measuredesc|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+--------------+-----------------+-------------+-------------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|            01|cs_basemeasureid1

In [8]:
#
df = spark.read.load('D:\SpectraMD\Feed\Benchmarks',format='csv',sep='|',header=True,schema=( 'Year string,State  string,     LOB string,Benchmark_category string,source string, averageName string,5thpercentile string,10thpercentile string,25thPercentile string,33_33thPercentile string,50thPercentile string,66_67thPercentile int,75thPercentile int,90thPercentile int'))
df.show()

#To Select all the columns
df.select(col("*")).show()

#alias 
df.select(col('Year').alias('Year2')).show()

#using the orderby function
df.orderBy(col('5thpercentile').asc()).show()
# fetching the specific column 
df.orderBy(col('5thpercentile').asc()).select('source').show()

#Cast funtion
df.select(df.State.cast("string")).show()

#between function
df.where(col("25thPercentile").between(252,253)).show()

# contains
df.where(col("State").contains('FL')).show()

#like function
df.where(col("State").like('%FL%')).show()

#isin is use for multiple vales




+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|
|HEDIS_2022|   FL|Medicaid|Benchmarkcategory2| test2|averageName2|           52|           102|           252|              332|           502|              662|           752|           902|
|HEDIS_2021|   VA|Medicaid|Benchmarkcate